# Before your start:
- Read the README.md file
- Comment as much as you can and use the resources (README.md file)
- Happy learning!

In [50]:
# import numpy and pandas
import numpy as np 
import pandas as pd 



# Challenge 1 - Exploring the Data

In this challenge, we will examine all salaries of employees of the City of Chicago. We will start by loading the dataset and examining its contents.

In [51]:
employees = pd.read_csv('Current_Employee_Names__Salaries__and_Position_Titles.csv')

Examine the `salaries` dataset using the `head` function below.

In [52]:
employees.head()

,Name,Job Titles,Department,Full or Part-Time,Salary or Hourly,Typical Hours,Annual Salary,Hourly Rate
0,"AARON, JEFFERY M",SERGEANT,POLICE,F,Salary,NaN,101442.0,NaN
1,"AARON, KARINA",POLICE OFFICER (ASSIGNED AS DETECTIVE),POLICE,F,Salary,NaN,94122.0,NaN
2,"AARON, KIMBERLEI R",CHIEF CONTRACT EXPEDITER,GENERAL SERVICES,F,Salary,NaN,101592.0,NaN
3,"ABAD JR, VICENTE M",CIVIL ENGINEER IV,WATER MGMNT,F,Salary,NaN,110064.0,NaN
4,"ABASCAL, REECE E",TRAFFIC CONTROL AIDE-HOURLY,OEMC,P,Hourly,20.0,NaN,19.86


We see from looking at the `head` function that there is quite a bit of missing data. Let's examine how much missing data is in each column. Produce this output in the cell below

In [53]:
employees.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 33183 entries, 0 to 33182
Data columns (total 8 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Name               33183 non-null  object 
 1   Job Titles         33183 non-null  object 
 2   Department         33183 non-null  object 
 3   Full or Part-Time  33183 non-null  object 
 4   Salary or Hourly   33183 non-null  object 
 5   Typical Hours      8022 non-null   float64
 6   Annual Salary      25161 non-null  float64
 7   Hourly Rate        8022 non-null   float64
dtypes: float64(3), object(5)
memory usage: 1.4+ MB


Let's also look at the count of hourly vs. salaried employees. Write the code in the cell below

In [54]:
h = employees['Hourly Rate'].count()
s = employees['Annual Salary'].count()

print(f'There are {h} hourly employees and {s} salaried, so there are {h+s} in total')


There are 8022 hourly employees and 25161 salaried, so there are 33183 in total


What this information indicates is that the table contains information about two types of employees - salaried and hourly. Some columns apply only to one type of employee while other columns only apply to another kind. This is why there are so many missing values. Therefore, we will not do anything to handle the missing values.

There are different departments in the city. List all departments and the count of employees in each department.

# Challenge 2 - Hypothesis Tests

In this section of the lab, we will test whether the hourly wage of all hourly workers is significantly different from $30/hr. Import the correct one sample test function from scipy and perform the hypothesis test for a 95% two sided confidence interval.

In [57]:
# Compare a list with a single value
# ttest_1samp(a, popmean, axis=0, nan_policy='propagate')
    # a - our sample
    # popmean - the value we want to test our sample against
    # axis - 
    # nan_policy - how to handle NaNs
        # 'propagate' returns nan if there are nans
        # 'raise' returns an error if there are nans
        # 'omit' ignore the nans and calculate using the existing numbers
from scipy.stats import ttest_1samp

statistic, p_value = ttest_1samp(employees['Hourly Rate'], 30, nan_policy='omit')
p_value_threshold = 1 - 0.95 # for 95% confidence interval
#print(statistic, p_value)
print(p_value < p_value_threshold)

# p_value < p_value_threshold:
    # True - means the hypothesis is within our confidence interval, hence is close to correct
    # False - means the hypothesis is not within our confidence interval, hence is almost centainly incorrect

True


We are also curious about salaries in the police force. The chief of police in Chicago claimed in a press briefing that salaries this year are higher than last year's mean of $86000/year a year for all salaried employees. Test this one sided hypothesis using a 95% confidence interval.

Hint: A one tailed test has a p-value that is half of the two tailed p-value. If our hypothesis is greater than, then to reject, the test statistic must also be positive.

In [58]:
# Comparing a list with 1 value but returning only 1 side  -> one-tailed t-test
# ttest_1samp(a, popmean, nan_policy='propagate')

statistic, p_value = ttest_1samp(employees['Annual Salary'], 86000, nan_policy='omit')

p_value_threshold = 1 - 0.95 # for 95% confidence interval

# since we are only interested with the values higher (just one tail), we divide by 2

#print(statistic, p_value)
print(p_value < p_value_threshold / 2)

True


Using the `crosstab` function, find the department that has the most hourly workers. 

In [59]:
pd.crosstab(employees['Department'],employees['Salary or Hourly']).sort_values('Hourly', ascending=False).head(1)


Salary or Hourly,Hourly,Salary
Department,,
STREETS & SAN,1862,336


The workers from the department with the most hourly workers have complained that their hourly wage is less than $35/hour. Using a one sample t-test, test this one-sided hypothesis at the 95% confidence level.

In [60]:
# Comparing a list with 1 value but returning only 1 side  -> one-tailed t-test
statistic, p_value = ttest_1samp(employees[employees['Department'] == 'STREETS & SAN']['Hourly Rate'], 35, nan_policy='omit')

p_value_threshold = 1 - 0.95 # for 95% confidence interval

# since we are only interested with the  values lower (just one tail), we divide by 2
#print(statistic, p_value)
print(p_value < p_value_threshold / 2)

True


# Challenge 3: To practice - Constructing Confidence Intervals

While testing our hypothesis is a great way to gather empirical evidence for accepting or rejecting the hypothesis, another way to gather evidence is by creating a confidence interval. A confidence interval gives us information about the true mean of the population. So for a 95% confidence interval, we are 95% sure that the mean of the population is within the confidence interval. 
).

To read more about confidence intervals, click [here](https://en.wikipedia.org/wiki/Confidence_interval).


In the cell below, we will construct a 95% confidence interval for the mean hourly wage of all hourly workers. 

The confidence interval is computed in SciPy using the `t.interval` function. You can read more about this function [here](https://docs.scipy.org/doc/scipy-0.14.0/reference/generated/scipy.stats.t.html).

To compute the confidence interval of the hourly wage, use the 0.95 for the confidence level, number of rows - 1 for degrees of freedom, the mean of the sample for the location parameter and the standard error for the scale. The standard error can be computed using [this](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.sem.html) function in SciPy.

In [96]:
from scipy.stats import t

mean = employees['Hourly Rate'].dropna().mean()
std_dev = employees['Hourly Rate'].dropna().std()

# t.interval(alpha, df, loc, scale)
    # alpha - confidence level
    # df - degrees of freedom -> len - 1
    # loc - mean
    # scale - standard deviation (because it's the entire pop)

conf_interval = t.interval(0.95, len(employees['Hourly Rate'].dropna())-1, mean, std_dev) # because it's the entire pop

conf_interval

(9.04476857882128, 56.5323468537392)

Now construct the 95% confidence interval for all salaried employeed in the police in the cell below.

In [95]:
# Your code here:
mean = employees['Annual Salary'].dropna().mean()
std_dev = employees['Annual Salary'].dropna().std()

# t.interval(alpha, df, loc, scale)
    # alpha - confidence level
    # df - degrees of freedom -> len - 1
    # loc - mean
    # scale - standard deviation (because it's the entire pop)

conf_interval = t.interval(0.95, len(employees['Annual Salary'].dropna())-1, mean, std_dev) # because it's the entire pop

conf_interval


(45544.71855637843, 128029.28102392443)

# Bonus Challenge - Hypothesis Tests of Proportions

Another type of one sample test is a hypothesis test of proportions. In this test, we examine whether the proportion of a group in our sample is significantly different than a fraction. 

You can read more about one sample proportion tests [here](http://sphweb.bumc.bu.edu/otlt/MPH-Modules/BS/SAS/SAS6-CategoricalData/SAS6-CategoricalData2.html).

In the cell below, use the `proportions_ztest` function from `statsmodels` to perform a hypothesis test that will determine whether the number of hourly workers in the City of Chicago is significantly different from 25% at the 95% confidence level.

In [63]:
# Your code here:

